In [26]:
import numpy as np
import gmpy2

# def random_20d_lattice_basis(low=-100000, high=100001):
#     while True:
#         basis = np.random.randint(low, high, size=(20, 20))
#         if np.linalg.matrix_rank(basis) == 20:
#             return basis
#
# # 示例：生成一个二十维格基
# basis = random_20d_lattice_basis()
# print(basis)

In [27]:
def gram_schmidt(B):
    n = len(B)
    B_star = [[gmpy2.mpz(0) for _ in range(len(B[0]))] for _ in range(n)]
    mu = [[gmpy2.mpz(0) for _ in range(n)] for _ in range(n)]

    for i in range(n):
        B_star[i] = B[i][:]
        for j in range(i):
            mu[i][j] = sum(gmpy2.mpz(B[i][k]) * gmpy2.mpz(B_star[j][k]) for k in range(len(B[0]))) // sum(gmpy2.mpz(B_star[j][k]) * gmpy2.mpz(B_star[j][k]) for k in range(len(B[0])))
            for k in range(len(B[0])):
                B_star[i][k] -= mu[i][j] * B_star[j][k]

    return B_star, mu

def lll_reduction(B, delta=0.75):
    n = len(B)
    B = [[gmpy2.mpz(x) for x in row] for row in B]
    k = 1
    B_star, mu = gram_schmidt(B)

    while k < n:
        for j in range(k - 1, -1, -1):
            c = gmpy2.mpz(round(mu[k][j]))
            if c != 0:
                for i in range(len(B[0])):
                    B[k][i] -= c * B[j][i]
                B_star, mu = gram_schmidt(B)

        lhs = sum(gmpy2.mpz(B_star[k][i]) * gmpy2.mpz(B_star[k][i]) for i in range(len(B[0])))
        rhs = (delta - mu[k][k - 1] ** 2) * sum(gmpy2.mpz(B_star[k - 1][i]) * gmpy2.mpz(B_star[k - 1][i]) for i in range(len(B[0])))

        if lhs >= rhs:
            k += 1
        else:
            B[k], B[k - 1] = B[k - 1], B[k]
            B_star, mu = gram_schmidt(B)
            k = max(k - 1, 1)

    return B

# # 示例：生成一个随机二十维格基并进行LLL约减
# basis = random_20d_lattice_basis()
# print("原始二十维格基：\n", basis)
# reduced_basis = lll_reduction(basis)
# print("LLL约减后的二十维格基：\n", reduced_basis)
#
# # 打印 LLL 规约后的第一个向量
# print("LLL规约后的第一个向量：", reduced_basis[0])
# # 打印 LLL 规约后的第一个向量的范数
# print("LLL规约后的第一个向量的范数：", np.linalg.norm(reduced_basis[0]))


In [28]:
def read_lattice_basis(file_path):
    """
    从文件中读取格基。

    参数:
    - file_path: 文件路径 (str)

    返回:
    - B: 格基 (numpy.ndarray)
    """
    with open(file_path, 'r') as f:
        data = f.read()

    # 去掉多余的方括号并按行分割
    data = data.replace('[', '').replace(']', '').strip()
    lines = data.split('\n')

    # 将每行转换为整数列表
    B = [list(map(int, line.split())) for line in lines if line.strip()]

    return np.array(B)

# 读取 40 维格基文件
file_path = 'svpchallengedim40seed0.txt'
basis_40d = read_lattice_basis(file_path)

# 使用 LLL 算法规约
reduced_basis_40d = lll_reduction(basis_40d)
print("LLL约减后的40维格基：")
print(reduced_basis_40d)


LLL约减后的40维格基：
[[mpz(1032017880633877711915629194277186147739724330193206020300230159022029956161177352932320), mpz(441245161665606727613883346300275739361517660652402978079510618516836658799476052211167), mpz(20363685626068990720613603573612095537892944243096721121411254), mpz(75961), mpz(0), mpz(0), mpz(0), mpz(0), mpz(0), mpz(0), mpz(0), mpz(0), mpz(0), mpz(0), mpz(0), mpz(0), mpz(0), mpz(0), mpz(0), mpz(0), mpz(0), mpz(0), mpz(0), mpz(0), mpz(0), mpz(0), mpz(0), mpz(0), mpz(0), mpz(0), mpz(0), mpz(0), mpz(0), mpz(0), mpz(0), mpz(0), mpz(0), mpz(0), mpz(0), mpz(0)], [mpz(1259816839544140623158642952238448962556677742582605411394441990971082740536426764103123), mpz(-1721054211879902745762912357769622524844711424081039191992347244389759273320673722823603), mpz(-79427515496938859529032525822421227869145325373558126779039471), mpz(-296282), mpz(0), mpz(0), mpz(0), mpz(0), mpz(0), mpz(0), mpz(0), mpz(0), mpz(0), mpz(0), mpz(0), mpz(0), mpz(0), mpz(0), mpz(0), mpz(0), mpz(0), mpz(0), mpz(0

In [29]:
# def sieving_algorithm(B):
#     """
#     筛选算法，用于从格基中筛选满足条件的向量。
#
#     参数:
#     - B: 格基 (numpy.ndarray)
#     - R: 饱和半径 (float)
#
#     返回:
#     - L: 筛选后的格向量列表
#     """
#     # 初始化 L 为一组随机格向量
#     n = B.shape[0]
#     num_vectors = int(2 ** (0.2075 * n + np.log2(n)))  # 计算 2^{0.2075n+o(n)} 的近似值
#     L = []
#     for _ in range(num_vectors):
#         L.append(np.dot(np.random.randint(-100000, 100001, size=n), B))
#
#     while True:
#         saturated = True
#         for i in range(len(L)):
#             for j in range(i + 1, len(L)):
#                 u, v = L[i], L[j]
#                 diff = u - v
#                 if np.linalg.norm(diff) < max(np.linalg.norm(w) for w in L):
#                     # 替换 L 中最长的向量
#                     longest_idx = np.argmax([np.linalg.norm(w) for w in L])
#                     L[longest_idx] = diff
#                     saturated = False
#         # 检查是否饱和球体
#         if saturated:
#             break
#
#     # 返回筛选后的向量列表
#     return L


# filtered_vectors = sieving_algorithm(reduced_basis)
# print("筛选后的格向量：")
# for vec in filtered_vectors:
#     print(vec)

